In [ ]:
!pip install confluent-kafka==2.2.0

In [ ]:
!pip install pandas numpy requests redis neo4j faiss-cpu torch transformers scikit-learn llama-index hdbscan opentelemetry-api google-cloud-spanner confluent-kafka


In [ ]:
!pip install pandas numpy requests redis neo4j faiss-cpu torch transformers scikit-learn llama-index hdbscan opentelemetry-api google-cloud-spanner confluent-kafka


In [ ]:
!pip install timezonefinder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import uuid

# Sample dataset paths (replace with actual paths)
dataset_paths = {
    "call_records": "/content/call_records.csv",
    "ip_records": "/content/ip_records.csv",
    "spy_records": "/content/spy_records.csv",
    "criminal_records": "/content/criminal_records.csv",
    "village_data": "/content/village_records.csv",
    "social_media": "/content/social_media_records.csv"
}

# Define rules for identifying criminals
rules = {
    "call_records": lambda df: df[df["call_type"].str.lower() == "international"] if "call_type" in df.columns else None,
    "ip_records": lambda df: df[df["activity"].str.contains("dark web|vpn|fishy site", case=False, na=False)] if "activity" in df.columns else None,
    "spy_records": lambda df: df[df["suspicion_score"] > 0.8] if "suspicion_score" in df.columns else None,
    "criminal_records": lambda df: df,  # Already known criminals
    "village_data": lambda df: df[df["border_crossing"] == True] if "border_crossing" in df.columns else None,
    "social_media": lambda df: df[df["keywords"].str.contains("illegal|suspicious", case=False, na=False)] if "keywords" in df.columns else None
}

# Function to load datasets dynamically
def load_dataset(path):
    try:
        if path.endswith(".csv"):
            return pd.read_csv(path)
        elif path.endswith(".json"):
            return pd.read_json(path)
        elif path.endswith(".txt"):
            return pd.read_csv(path, delimiter="\t", header=None)  # Assuming tab-separated
    except Exception as e:
        print(f"Error loading {path}: {e}")
    return None

# Process datasets and find suspicious individuals
suspects = []
for dataset, path in dataset_paths.items():
    df = load_dataset(path)
    if df is not None and dataset in rules:
        if df.empty:
            print(f"Warning: {dataset} is empty. Skipping.")
            continue

        filtered_df = rules[dataset](df)
        if filtered_df is None:
            print(f"Warning: Required column missing in {dataset}. Available columns: {df.columns.tolist()}")
            continue

        for _, row in filtered_df.iterrows():
            suspect = {
                "log_id": str(uuid.uuid4()),
                "name": row.get("name", "Unknown"),
                "reason": f"Flagged in {dataset.replace('_', ' ').title()} dataset",
                "dataset_match": [dataset],
                "risk_score": round(0.8 + 0.2 * ("suspicion_score" in row and row["suspicion_score"]), 2)
            }
            suspects.append(suspect)

# Save results as JSON
output_file = "suspected_criminals.json"
with open(output_file, "w") as f:
    json.dump(suspects, f, indent=4)

print(f"Processed datasets. Identified {len(suspects)} suspicious individuals. Results saved to {output_file}.")


Processed datasets. Identified 100 suspicious individuals. Results saved to suspected_criminals.json.


In [ ]:
import pandas as pd
import json
import uuid

# Sample dataset paths (replace with actual paths)
dataset_paths = {
    "call_records": "/content/call_records.csv",
    "ip_records": "/content/ip_records.csv",
    "spy_records": "/content/spy_records.csv",
    "criminal_records": "/content/criminal_records.csv",
    "village_data": "/content/village_records.csv",
    "social_media": "/content/social_media_records.csv"
}

# Function to load datasets dynamically
def load_dataset(path):
    if path.endswith(".csv"):
        return pd.read_csv(path)
    elif path.endswith(".json"):
        return pd.read_json(path)
    elif path.endswith(".txt"):
        return pd.read_csv(path, delimiter="\t", header=None)  # Assuming tab-separated
    return None

# Load all datasets
datasets = {}
for key, path in dataset_paths.items():
    df = load_dataset(path)
    if df is not None:
        datasets[key] = df
        print(f"Loaded {key} with columns: {df.columns.tolist()}")  # Debugging column names

# Function to determine if a person is a suspect
def investigate_person(person_name):
    suspect = {
        "id": str(uuid.uuid4()),
        "name": person_name,
        "reason": "",
        "dataset_match": [],
        "risk_score": 0.0
    }

    # Check if the person is in the village data
    village_df = datasets.get("village_data", pd.DataFrame())
    in_village = village_df[village_df["Head Name"].str.contains(person_name, na=False, case=False)]
    if in_village.empty:
        suspect["reason"] += "Not found in village records. "
        suspect["risk_score"] += 0.2
    else:
        suspect["reason"] += "Found in village records. "
        suspect["dataset_match"].append("village_data")

    # Check if the person is in the criminal records
    criminal_df = datasets.get("criminal_records", pd.DataFrame())
    in_criminal = criminal_df[criminal_df["Name"].str.contains(person_name, na=False, case=False)]
    if not in_criminal.empty:
        suspect["reason"] += "Listed in criminal records. HIGH RISK. "
        suspect["dataset_match"].append("criminal_records")
        suspect["risk_score"] += 1.0  # Highest risk

    # Check call records for international calls
    call_df = datasets.get("call_records", pd.DataFrame())
    if not call_df.empty and "Sender" in call_df.columns:
        calls_made = call_df[(call_df["Sender"] == person_name) & (call_df["International_Call"] == "Yes")]
        if not calls_made.empty:
            suspect["reason"] += "Made international calls. "
            suspect["dataset_match"].append("call_records")
            suspect["risk_score"] += 0.3

    # Check IP records for suspicious activity
    ip_df = datasets.get("ip_records", pd.DataFrame())
    if not ip_df.empty and "Sender" in ip_df.columns:
        ip_usage = ip_df[(ip_df["Sender"] == person_name) & (ip_df["VPN_Used"] == "Yes")]
        if not ip_usage.empty:
            suspect["reason"] += "Used VPN or accessed suspicious websites. "
            suspect["dataset_match"].append("ip_records")
            suspect["risk_score"] += 0.5

    # Check spy records for suspicious communication
    spy_df = datasets.get("spy_records", pd.DataFrame())
    if not spy_df.empty and "Sender" in spy_df.columns:
        spy_mentions = spy_df[spy_df["Sender"] == person_name]
        if not spy_mentions.empty:
            suspect["reason"] += "Mentioned in spy records. "
            suspect["dataset_match"].append("spy_records")
            suspect["risk_score"] += 0.7

    # Check social media activity
    social_df = datasets.get("social_media", pd.DataFrame())
    if not social_df.empty and "User_Name" in social_df.columns:
        suspicious_posts = social_df[(social_df["User_Name"] == person_name) & (social_df["Suspicious_Activity"].notna())]
        if not suspicious_posts.empty:
            suspect["reason"] += "Suspicious activity detected on social media. "
            suspect["dataset_match"].append("social_media")
            suspect["risk_score"] += 0.4

    # Final decision
    if suspect["risk_score"] > 0:
        return suspect
    return None

# Identifying suspects from all datasets
suspects = []
for key, df in datasets.items():
    name_column = next((col for col in df.columns if "name" in col.lower() or "sender" in col.lower()), None)
    if name_column:
        for person_name in df[name_column].dropna().unique():
            suspect = investigate_person(person_name)
            if suspect:
                suspects.append(suspect)
                print(f"Flagged: {suspect}")  # Debugging flagged individuals

# Save results as JSON
output_file = "suspected_criminals.json"
with open(output_file, "w") as f:
    json.dump(suspects, f, indent=4)

print(f"Processed datasets. Identified {len(suspects)} suspicious individuals. Results saved to {output_file}.")


Loaded call_records with columns: ['Call_ID', 'Sender', 'Receiver', 'Call_Duration', 'Call_Location', 'International_Call', 'Timestamp']
Loaded ip_records with columns: ['IP_ID', 'Sender', 'Receiver', 'IP_Address', 'VPN_Used', 'Website_Accessed', 'Timestamp']
Loaded spy_records with columns: ['Spy_ID', 'Sender', 'Receiver', 'Voice_Sample', 'Identified_Speaker', 'Timestamp']
Loaded criminal_records with columns: ['Criminal_ID', 'Name', 'Crime_Type', 'Last_Known_Location', 'Under_Surveillance']
Loaded village_data with columns: ['Family ID', 'Head Name', 'Family Members', 'Village Name', 'Border Area', 'Suspected Activity']
Loaded social_media with columns: ['Social_ID', 'User_Name', 'Platform', 'Suspicious_Activity', 'Last_Post_Location']
Flagged: {'id': '2996495c-87d0-494f-bda1-8910cc46173e', 'name': 'Suraj Thakur', 'reason': 'Not found in village records. Listed in criminal records. HIGH RISK. Made international calls. Used VPN or accessed suspicious websites. Mentioned in spy records

In [ ]:
import pandas as pd
import json
import uuid
from collections import defaultdict

# Sample dataset paths (replace with actual paths)
dataset_paths = {
    "call_records": "/content/call_records.csv",
    "ip_records": "/content/ip_records.csv",
    "spy_records": "/content/spy_records.csv",
    "criminal_records": "/content/criminal_records.csv",
    "village_data": "/content/village_records.csv",
    "social_media": "/content/social_media_records.csv"
}

# Function to load datasets dynamically
def load_dataset(path):
    if path.endswith(".csv"):
        return pd.read_csv(path)
    elif path.endswith(".json"):
        return pd.read_json(path)
    elif path.endswith(".txt"):
        return pd.read_csv(path, delimiter="\t", header=None)  # Assuming tab-separated
    return None

# Load all datasets
datasets = {}
for key, path in dataset_paths.items():
    df = load_dataset(path)
    if df is not None:
        datasets[key] = df
        print(f"Loaded {key} with columns: {df.columns.tolist()}")  # Debugging column names

# Function to determine if a person is a suspect
def investigate_person(person_name):
    suspect = {
        "id": str(uuid.uuid4()),
        "name": person_name,
        "reason": "",
        "dataset_match": [],
        "risk_score": 0.0
    }

    # Check if the person is in the village data
    village_df = datasets.get("village_data", pd.DataFrame())
    in_village = village_df[village_df["Head Name"].str.contains(person_name, na=False, case=False)]
    if in_village.empty:
        suspect["reason"] += "Not found in village records. "
        suspect["risk_score"] += 0.2
    else:
        count = len(in_village)
        suspect["reason"] += f"Found {count} times in village records. "
        suspect["dataset_match"].append("village_data")
        suspect["risk_score"] += 0.2 * count

    # Check if the person is in the criminal records
    criminal_df = datasets.get("criminal_records", pd.DataFrame())
    in_criminal = criminal_df[criminal_df["Name"].str.contains(person_name, na=False, case=False)]
    if not in_criminal.empty:
        count = len(in_criminal)
        suspect["reason"] += f"Listed {count} times in criminal records. HIGH RISK. "
        suspect["dataset_match"].append("criminal_records")
        suspect["risk_score"] += 1.0 * count  # Highest risk

    # Check call records for international calls
    call_df = datasets.get("call_records", pd.DataFrame())
    if not call_df.empty and "Sender" in call_df.columns:
        calls_made = call_df[(call_df["Sender"] == person_name) & (call_df["International_Call"] == "Yes")]
        if not calls_made.empty:
            count = len(calls_made)
            suspect["reason"] += f"Made {count} international calls. "
            suspect["dataset_match"].append("call_records")
            suspect["risk_score"] += 0.3 * count

    # Check IP records for suspicious activity
    ip_df = datasets.get("ip_records", pd.DataFrame())
    if not ip_df.empty and "Sender" in ip_df.columns:
        ip_usage = ip_df[(ip_df["Sender"] == person_name) & (ip_df["VPN_Used"] == "Yes")]
        if not ip_usage.empty:
            count = len(ip_usage)
            suspect["reason"] += f"Used VPN or accessed suspicious websites {count} times. "
            suspect["dataset_match"].append("ip_records")
            suspect["risk_score"] += 0.5 * count

    # Check spy records for suspicious communication
    spy_df = datasets.get("spy_records", pd.DataFrame())
    if not spy_df.empty and "Sender" in spy_df.columns:
        spy_mentions = spy_df[spy_df["Sender"] == person_name]
        if not spy_mentions.empty:
            count = len(spy_mentions)
            suspect["reason"] += f"Mentioned {count} times in spy records. "
            suspect["dataset_match"].append("spy_records")
            suspect["risk_score"] += 0.7 * count

    # Check social media activity
    social_df = datasets.get("social_media", pd.DataFrame())
    if not social_df.empty and "User_Name" in social_df.columns:
        suspicious_posts = social_df[(social_df["User_Name"] == person_name) & (social_df["Suspicious_Activity"] == "Yes")]
        if not suspicious_posts.empty:
            count = len(suspicious_posts)
            suspect["reason"] += f"Suspicious activity detected {count} times on social media. "
            suspect["dataset_match"].append("social_media")
            suspect["risk_score"] += 0.4 * count

    return suspect

# Identifying suspects from all datasets
suspects = defaultdict(lambda: {"risk_score": 0, "reasons": []})
for key, df in datasets.items():
    name_column = next((col for col in df.columns if "name" in col.lower() or "sender" in col.lower()), None)
    if name_column:
        for person_name in df[name_column].dropna().unique():
            suspect = investigate_person(person_name)
            if suspect["risk_score"] > 0:
                suspects[person_name]["risk_score"] += suspect["risk_score"]
                suspects[person_name]["reasons"].append(suspect["reason"])

# Convert to list and sort by risk score
sorted_suspects = sorted(
    [{"name": k, "risk_score": v["risk_score"], "reasons": v["reasons"]} for k, v in suspects.items()],
    key=lambda x: x["risk_score"],
    reverse=True
)

# Save results as JSON
output_file = "suspected_criminals_ranked.json"
with open(output_file, "w") as f:
    json.dump(sorted_suspects, f, indent=4)

print(f"Processed datasets. Identified {len(sorted_suspects)} suspicious individuals. Results saved to {output_file}.")

# Print top 10 suspects
print("\nTop 10 Suspects:")
for i, suspect in enumerate(sorted_suspects[:10], 1):
    print(f"{i}. {suspect['name']} - Risk Score: {suspect['risk_score']:.2f}")
    print(f"   Reasons: {' '.join(suspect['reasons'])}")

In [ ]:
import pandas as pd

# Load the datasets
call_records = pd.read_csv("call_records.csv")
ip_records = pd.read_csv("ip_records.csv")
spy_records = pd.read_csv("spy_records.csv")
criminal_records = pd.read_csv("criminal_records.csv")
village_data = pd.read_csv("village_records.csv")
social_media = pd.read_csv("social_media_records.csv")
financial_records = pd.read_csv("financial_records.csv")
travel_history = pd.read_csv("travel_history.csv")
gps_records = pd.read_csv("gps_records.csv")

# Get the unique set of people from relevant columns in all datasets
people = set()

# Datasets with "Sender" and "Receiver" columns (communication data)
communication_datasets = {
    "call_records": (call_records, "Sender", "Receiver"),
    "ip_records": (ip_records, "Sender", "Receiver"),
    "spy_records": (spy_records, "Sender", "Receiver")
}

# Iterate through the communication datasets to extract people
for dataset_name, (df, sender_col, receiver_col) in communication_datasets.items():
    people.update(df[sender_col].unique())
    people.update(df[receiver_col].unique())

# Datasets with only one name column
single_name_datasets = {
    "criminal_records": (criminal_records, "Name"),
    "financial_records": (financial_records, "Person_Name"),
    "travel_history": (travel_history, "Person_Name"),
    "gps_records": (gps_records, "Person_Name")
}

# Iterate through the single name datasets to extract people
for dataset_name, (df, name_col) in single_name_datasets.items():
    people.update(df[name_col].unique())

# Create a dictionary to store relationships
relationships = {person: {} for person in people}

# Function to find relationships in communication datasets
def find_communication_relationships(relationships, df, sender_col, receiver_col, dataset_name):
    for index, row in df.iterrows():
        sender = row[sender_col]
        receiver = row[receiver_col]

        if sender in relationships and receiver in relationships:
            if receiver not in relationships[sender]:
                relationships[sender][receiver] = [dataset_name]
            else:
                relationships[sender][receiver].append(dataset_name)
    return relationships

# Iterate through communication datasets
for dataset_name, (df, sender_col, receiver_col) in communication_datasets.items():
    relationships = find_communication_relationships(relationships, df, sender_col, receiver_col, dataset_name)

# Print the relationships
for person, related_people in relationships.items():
    print(f"Relationships for {person}:")
    for related_person, datasets in related_people.items():
        print(f"  - {related_person}: {', '.join(datasets)}")


In [ ]:
import pandas as pd
from collections import defaultdict

# Load the datasets
call_records = pd.read_csv('call_records.csv')
ip_records = pd.read_csv('ip_records.csv')

# Extract unique individuals from call records and IP records
individuals = set(call_records['Sender']).union(set(call_records['Receiver'])).union(
    set(ip_records['Sender']).union(set(ip_records['Receiver']))
)

# Initialize a dictionary to store relationships
relationships = defaultdict(lambda: defaultdict(list))

# Process call records
for _, row in call_records.iterrows():
    sender = row['Sender']
    receiver = row['Receiver']
    details = {
        'Type': 'Call',
        'Duration': row['Call_Duration'],
        'Location': row['Call_Location'],
        'International': row['International_Call'],
        'Timestamp': row['Timestamp']
    }
    relationships[sender][receiver].append(details)
    relationships[receiver][sender].append(details)  # Add reverse relationship

# Process IP records
for _, row in ip_records.iterrows():
    sender = row['Sender']
    receiver = row['Receiver']
    details = {
        'Type': 'IP',
        'IP_Address': row['IP_Address'],
        'VPN_Used': row['VPN_Used'],
        'Website_Accessed': row['Website_Accessed'],
        'Timestamp': row['Timestamp']
    }
    relationships[sender][receiver].append(details)
    relationships[receiver][sender].append(details)  # Add reverse relationship

# Generate a table of relationships between all individuals
relationship_table = []
for person in individuals:
    for related_person, details in relationships[person].items():
        total_calls = sum(1 for d in details if d['Type'] == 'Call')
        total_ip_connections = sum(1 for d in details if d['Type'] == 'IP')
        relationship_table.append({
            'Person': person,
            'Related Person': related_person,
            'Total Calls': total_calls,
            'Total IP Connections': total_ip_connections,
            'Details': details
        })

# Convert the relationship table into a DataFrame for better visualization
relationship_df = pd.DataFrame(relationship_table)

# Save the relationship table to a CSV file
relationship_df.to_csv('relationship_table.csv', index=False)

# Display the first few rows of the table
print(relationship_df.head())

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Function to load dataset dynamically
def load_dataset(path):
    if path.endswith(".csv"):
        return pd.read_csv(path)
    elif path.endswith(".json"):
        return pd.read_json(path)
    elif path.endswith(".txt"):
        return pd.read_csv(path, delimiter="\t", header=None)  # Assuming tab-separated
    return None

# Path to the social media dataset (replace with actual path)
social_media_path = "/content/updated_social_network_connections.csv"
social_df = load_dataset(social_media_path)

# Identify relevant columns dynamically
columns = social_df.columns

# Find columns that contain user relationship information
sender_col = next((col for col in columns if "sender" in col.lower() or "user" in col.lower()), None)
receiver_col = next((col for col in columns if "receiver" in col.lower() or "follower" in col.lower()), None)

if not sender_col or not receiver_col:
    raise ValueError("Could not detect appropriate sender/receiver columns in the dataset")

# Extract suspect interactions
interactions = social_df[[sender_col, receiver_col]].dropna()

# Build a directed graph
G = nx.DiGraph()
for _, row in interactions.iterrows():
    sender, receiver = row[sender_col], row[receiver_col]
    if G.has_edge(sender, receiver):
        G[sender][receiver]['weight'] += 1  # Increase weight for repeated interactions
    else:
        G.add_edge(sender, receiver, weight=1)

# Identify strong connections
threshold = 3  # Define a threshold for frequent interactions
strong_connections = [(u, v, d['weight']) for u, v, d in G.edges(data=True) if d['weight'] >= threshold]

# Label relationships
relationships = []
for u, v, weight in strong_connections:
    relation_type = "Middleman" if nx.has_path(G, v, u) else "Right-hand Man"
    relationships.append({
        "Suspect": u,
        "Connected To": v,
        "Interaction Count": weight,
        "Relationship": relation_type
    })

# Save relationships to a JSON file
output_file = "suspect_relationships.json"
pd.DataFrame(relationships).to_json(output_file, orient="records", indent=4)

# Visualize the network
def plot_network(graph):
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(graph)
    weights = [graph[u][v]['weight'] for u, v in graph.edges()]
    nx.draw(graph, pos, with_labels=True, node_size=2000, node_color="skyblue", edge_color=weights, width=2.0, cmap=plt.cm.Blues)
    plt.title("Suspect Interactions Network")
    plt.show()

plot_network(G)

print(f"Identified {len(relationships)} strong relationships. Results saved to {output_file}.")


In [ ]:
import pandas as pd
import networkx as nx
import json

# Load dataset
dataset_path = "social_media_transactions.csv"  # Update with the correct path
df = pd.read_csv(dataset_path)

# Ensure required columns exist
def validate_columns(df, required_cols):
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

validate_columns(df, ["Sender", "Receiver", "Relationship", "Transaction_Count"])

# Create a directed graph
G = nx.DiGraph()

# Add edges to the graph
for _, row in df.iterrows():
    sender, receiver, relation, count = row["Sender"], row["Receiver"], row["Relationship"], row["Transaction_Count"]
    G.add_edge(sender, receiver, relationship=relation, weight=count)

# Analyze relationships
def analyze_relationships(graph):
    suspect_network = {}

    for suspect in graph.nodes:
        connections = list(graph[suspect])  # Get outgoing connections
        suspect_network[suspect] = {
            "total_connections": len(connections),
            "connections": []
        }

        for conn in connections:
            edge_data = graph[suspect][conn]
            suspect_network[suspect]["connections"].append({
                "name": conn,
                "relationship": edge_data["relationship"],
                "interaction_count": edge_data["weight"]
            })

    return suspect_network

suspect_relations = analyze_relationships(G)

# Save results as JSON
output_file = "suspect_network.json"
with open(output_file, "w") as f:
    json.dump(suspect_relations, f, indent=4)

print(f"Processed dataset. Relationship network saved to {output_file}.")


In [ ]:
import pandas as pd
import networkx as nx
import json

# Load the dataset
df = pd.read_csv("social_media_suspects.csv")

# Create a graph
G = nx.DiGraph()

# Add nodes (Users)
for user in df["User"].unique():
    G.add_node(user)

# Add edges (Relationships based on interactions)
for _, row in df.iterrows():
    user = row["User"]
    following = row["Following"]
    interaction_count = row["Interaction_Count"]
    activity = row["Activity_Type"]

    # Define relationship strength based on interaction count
    if interaction_count > 30:
        relation = "Right-hand man"
    elif 15 <= interaction_count <= 30:
        relation = "Middleman"
    else:
        relation = "Weak Connection"

    # Add edge with attributes
    G.add_edge(user, following, weight=interaction_count, relationship=relation, activity=activity)

# Generate relationship report
relationships = []
for u, v, data in G.edges(data=True):
    relationships.append({
        "Suspect": u,
        "Connected_To": v,
        "Relationship": data["relationship"],
        "Interaction_Count": data["weight"],
        "Activity": data["activity"]
    })

# Save relationships as JSON
output_file = "social_media_relationships.json"
with open(output_file, "w") as f:
    json.dump(relationships, f, indent=4)

print(f"Relationship analysis saved to {output_file}")


In [ ]:
import pandas as pd
import networkx as nx
import json
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("social_media_suspects.csv")

# Create a graph
G = nx.DiGraph()

# Add nodes (Users)
for user in df["User"].unique():
    G.add_node(user)

# Add edges (Relationships based on interactions)
for _, row in df.iterrows():
    user = row["User"]
    following = row["Following"]
    interaction_count = row["Interaction_Count"]
    activity = row["Activity_Type"]

    # Define relationship strength based on interaction count
    if interaction_count > 30:
        relation = "Right-hand man"
    elif 15 <= interaction_count <= 30:
        relation = "Middleman"
    else:
        relation = "Weak Connection"

    # Add edge with attributes
    G.add_edge(user, following, weight=interaction_count, relationship=relation, activity=activity)

# Generate relationship report
relationships = []
for u, v, data in G.edges(data=True):
    relationships.append({
        "Suspect": u,
        "Connected_To": v,
        "Relationship": data["relationship"],
        "Interaction_Count": data["weight"],
        "Activity": data["activity"]
    })

# Save relationships as JSON
output_json = "social_media_relationships.json"
with open(output_json, "w") as f:
    json.dump(relationships, f, indent=4)

# Save relationships as CSV
output_csv = "social_media_relationships.csv"
pd.DataFrame(relationships).to_csv(output_csv, index=False)

print(f"Relationship analysis saved to {output_json} and {output_csv}")

# Draw the graph
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color="skyblue", edge_color="gray", font_size=8, node_size=1500)
plt.title("Social Media Suspect Network")
plt.show()


In [ ]:
import pandas as pd
import networkx as nx
import json
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Clean column names to remove extra spaces
df.columns = df.columns.str.strip()

# Print columns for debugging
print("Dataset Columns:", df.columns)

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15
}

# Create a graph
G = nx.DiGraph()

# Add nodes (Users)
for user in df["User"].unique():
    G.add_node(user)

# Add edges (Relationships based on interactions and scoring system)
for _, row in df.iterrows():
    user = row["User"]
    followings = row["Following"].split(", ") if isinstance(row["Following"], str) else []

    # Corrected column names
    interaction_count = row["Interaction_Count"]  # Updated column name
    activity = row["Activity_Type"]  # Updated column name
    score = interaction_count * activity_scores.get(activity, 1)  # Calculate score

    for following in followings:
        if following in df["User"].values:
            G.add_edge(user, following, weight=score, activity=activity)

# Generate relationship report
relationships = []
for u, v, data in G.edges(data=True):
    relationships.append({
        "Suspect": u,
        "Connected_To": v,
        "Score": data["weight"],
        "Activity": data["activity"]
    })

# Save relationships as JSON
json_output_file = "social_media_relationships.json"
with open(json_output_file, "w") as f:
    json.dump(relationships, f, indent=4)

# Save relationships as CSV
csv_output_file = "social_media_relationships.csv"
pd.DataFrame(relationships).to_csv(csv_output_file, index=False)

# Draw network graph
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=2000, font_size=8)
nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): data['weight'] for u, v, data in G.edges(data=True)}, font_size=6)
plt.title("Criminal Social Media Relationship Network")
plt.show()

print(f"Relationship analysis saved to {json_output_file} and {csv_output_file}")


In [ ]:
import pandas as pd
import networkx as nx
import json
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15
}

# Create a graph
G = nx.DiGraph()

# Add nodes (Users)
for user in df["User"].unique():
    G.add_node(user)

# Add edges (Relationships based on interactions and scoring system)
role_mapping = {}
for _, row in df.iterrows():
    user = row["User"]
    followings = row["Following"].split(", ") if isinstance(row["Following"], str) else []
    interaction_count = row["Interaction_Count"]
    activity = row["Activity_Type"]
    score = interaction_count * activity_scores.get(activity, 1)  # Calculate score

    for following in followings:
        if following in df["User"].values:
            G.add_edge(user, following, weight=score, activity=activity)

    # Assign roles based on interaction count
    if interaction_count > 30:
        role_mapping[user] = "Leader"
    elif 15 <= interaction_count <= 30:
        role_mapping[user] = "Right-hand Man"
    elif 5 <= interaction_count < 15:
        role_mapping[user] = "Middleman"
    else:
        role_mapping[user] = "Weak Link"

# Generate relationship report
relationships = []
for u, v, data in G.edges(data=True):
    relationships.append({
        "Suspect": u,
        "Connected_To": v,
        "Score": data["weight"],
        "Activity": data["activity"],
        "Role": role_mapping.get(u, "Unknown")
    })

# Save relationships as JSON
json_output_file = "social_media_relationships.json"
with open(json_output_file, "w") as f:
    json.dump(relationships, f, indent=4)

# Save relationships as CSV
csv_output_file = "social_media_relationships.csv"
pd.DataFrame(relationships).to_csv(csv_output_file, index=False)

# Draw network graph
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)

# Assign colors based on role
node_colors = []
for node in G.nodes():
    role = role_mapping.get(node, "Weak Link")
    if role == "Leader":
        node_colors.append("red")
    elif role == "Right-hand Man":
        node_colors.append("blue")
    elif role == "Middleman":
        node_colors.append("green")
    else:
        node_colors.append("gray")

nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='gray', node_size=2000, font_size=8)
nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): data['weight'] for u, v, data in G.edges(data=True)}, font_size=6)
plt.title("Criminal Relationship Network")
plt.show()

print(f"Relationship analysis saved to {json_output_file} and {csv_output_file}")


In [ ]:
import pandas as pd
import networkx as nx
import json
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Clean column names to remove extra spaces
df.columns = df.columns.str.strip()

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to visualize the hierarchy of connections for a specific person
def visualize_hierarchy(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Add the main person as a node
    G.add_node(person_name)

    # Get the row for the person
    person_row = df[df["User"] == person_name].iloc[0]

    # Get their following list and interaction details
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]
    activity_type = person_row["Activity_Type"]

    # Calculate score for each connection
    score = interaction_count * activity_scores.get(activity_type, 1)

    # Add edges for followings
    for following in followings:
        if following in df["User"].values:
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Recursively add connections for each following (2nd level hierarchy)
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            second_level_activity_type = following_row["Activity_Type"]
            second_level_score = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)

            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    G.add_edge(following, second_following, weight=second_level_score, activity=second_level_activity_type)

    # Draw the graph
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)

    # Draw nodes and edges
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=2000, font_size=8)

    # Add edge labels (weights/scores)
    edge_labels = {(u, v): f"{data['weight']} ({data['activity']})" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.title(f"Hierarchy of Connections for {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the person to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy(person_name)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy and sub-connections for a specific person
def visualize_hierarchy(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Add the main person as a node
    G.add_node(person_name)

    # Role mapping for coloring nodes
    role_mapping = {}

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]
    activity_type = person_row["Activity_Type"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for followings (first-level connections)
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            second_level_activity_type = following_row["Activity_Type"]

            # Assign role to the first-level connection
            role_mapping[following] = assign_roles(second_level_interaction_count)

            # Add edge between main person and first-level connection
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add edges for second-level connections (sub-connections)
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    third_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]

                    # Assign role to the second-level connection
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    # Draw the graph with proper coloring based on roles
    plt.figure(figsize=(12, 8))

    pos = nx.spring_layout(G, seed=42)

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='gray', node_size=2000, font_size=8)

    # Add edge labels (weights/scores)
    edge_labels = {(u, v): f"{data['weight']} ({data['activity']})" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.title(f"Hierarchy of Connections for {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy(person_name)


In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Prepare data for clustering
def prepare_data(df):
    # Create a mapping for users and their connections
    users = df["User"].unique()
    user_indices = {user: idx for idx, user in enumerate(users)}

    # Create a distance matrix based on connections
    n = len(users)
    distance_matrix = np.zeros((n, n))

    for _, row in df.iterrows():
        user_idx = user_indices[row["User"]]
        followings = row["Following"].split(", ") if isinstance(row["Following"], str) else []

        for following in followings:
            if following in user_indices:
                following_idx = user_indices[following]
                distance_matrix[user_idx, following_idx] = 1  # Assign connection strength

    return users, distance_matrix

users, distance_matrix = prepare_data(df)

# Perform hierarchical clustering using SciPy's linkage method
linkage_matrix = linkage(distance_matrix, method="ward")

# Plot dendrogram
plt.figure(figsize=(12, 8))
dendrogram(linkage_matrix, labels=users, orientation="top", leaf_rotation=90, leaf_font_size=10)
plt.title("Hierarchical Clustering of Criminal Connections")
plt.xlabel("Users")
plt.ylabel("Distance")
plt.show()



In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

In [ ]:
!pip install pygraphviz

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy in a tree-like structure
def visualize_hierarchy_tree(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Role mapping for coloring nodes
    role_mapping = {}

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]

            # Assign role to first-level connections
            role_mapping[following] = assign_roles(second_level_interaction_count)

            G.add_edge(person_name, following)

            # Add edges for second-level connections (sub-connections)
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]

                    # Assign role to second-level connections
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    G.add_edge(following, second_following)

    # Draw the graph using a hierarchical layout
    pos = nx.nx_agraph.graphviz_layout(G, prog="dot")  # Use Graphviz's dot layout for hierarchy

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy_tree(person_name)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import json

def load_dataset(file_path):
    """Load the dataset and ensure required columns exist."""
    df = pd.read_csv(file_path)

    # Check for required columns
    expected_columns = ['User', 'Platform', 'Followers', 'Following', 'Interaction_Count', 'Activity_Type']
    missing_cols = [col for col in expected_columns if col not in df.columns]

    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")

    return df

def build_network(df):
    """Construct a network graph from the dataset."""
    G = nx.DiGraph()  # Directed graph for hierarchy

    for _, row in df.iterrows():
        user = row['User']
        followings = str(row['Following']).split(',') if pd.notna(row['Following']) else []

        for following in followings:
            G.add_edge(user.strip(), following.strip())

    return G

def assign_roles(df):
    """Assign roles based on interaction count."""
    role_mapping = {}
    for _, row in df.iterrows():
        interaction_count = row['Interaction_Count']
        user = row['User']

        # Assign roles based on interaction count
        if interaction_count > 30:
            role_mapping[user] = "Leader"
        elif 15 <= interaction_count <= 30:
            role_mapping[user] = "Right-hand Man"
        elif 5 <= interaction_count < 15:
            role_mapping[user] = "Middleman"
        else:
            role_mapping[user] = "Weak Link"

    return role_mapping

def visualize_hierarchy(G, target_user, role_mapping):
    """Visualize the hierarchical structure."""
    if target_user not in G:
        raise ValueError(f"User '{target_user}' not found in dataset.")

    # Extract the ego graph (hierarchy of connections)
    hierarchy = nx.ego_graph(G, target_user, radius=2)

    # Use Graphviz's dot layout for hierarchical visualization
    pos = graphviz_layout(hierarchy, prog="dot")

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in hierarchy.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    # Draw the graph
    nx.draw(hierarchy, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=10)

    plt.title(f"Hierarchical View of {target_user}")
    plt.show()

def save_hierarchy_json(G, target_user, output_path):
    """Save the hierarchical structure as JSON."""
    hierarchy = nx.ego_graph(G, target_user, radius=2)

    hierarchy_data = {
        'target_user': target_user,
        'connections': list(hierarchy.edges)
    }

    with open(output_path, 'w') as f:
        json.dump(hierarchy_data, f, indent=4)

    print(f"Hierarchy saved to {output_path}")

# Main Execution
file_path = "/content/criminal_social_media_dataset.csv"
target_user = "Amit Verma"  # Change to desired user
output_json = "hierarchy_output.json"

df = load_dataset(file_path)
G = build_network(df)
role_mapping = assign_roles(df)
visualize_hierarchy(G, target_user, role_mapping)
save_hierarchy_json(G, target_user, output_json)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import json

def load_dataset(file_path):
    """Load the dataset and ensure required columns exist."""
    df = pd.read_csv(file_path)

    # Check for required columns
    expected_columns = ['User', 'Platform', 'Followers', 'Following', 'Interaction_Count', 'Activity_Type']
    missing_cols = [col for col in expected_columns if col not in df.columns]

    if missing_cols:
        raise ValueError(f"Missing columns in dataset: {missing_cols}")

    return df

def build_network(df):
    """Construct a network graph from the dataset."""
    G = nx.DiGraph()  # Directed graph for hierarchy

    for _, row in df.iterrows():
        user = row['User']
        followings = str(row['Following']).split(',') if pd.notna(row['Following']) else []

        for following in followings:
            G.add_edge(user.strip(), following.strip())

    return G

def assign_roles(df):
    """Assign roles based on interaction count."""
    role_mapping = {}
    for _, row in df.iterrows():
        interaction_count = row['Interaction_Count']
        user = row['User']

        # Assign roles based on interaction count
        if interaction_count > 30:
            role_mapping[user] = "Leader"
        elif 15 <= interaction_count <= 30:
            role_mapping[user] = "Right-hand Man"
        elif 5 <= interaction_count < 15:
            role_mapping[user] = "Middleman"
        else:
            role_mapping[user] = "Weak Link"

    return role_mapping

def visualize_hierarchy_tree(G, target_user, role_mapping):
    """Visualize the hierarchical structure."""
    if target_user not in G:
        raise ValueError(f"User '{target_user}' not found in dataset.")

    # Extract the ego graph (hierarchy of connections)
    hierarchy = nx.ego_graph(G, target_user, radius=2)  # Radius=2 includes direct and sub-connections

    # Use Graphviz's dot layout for hierarchical visualization
    pos = graphviz_layout(hierarchy, prog="dot")

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in hierarchy.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    # Draw the graph
    nx.draw(hierarchy, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=10)

    plt.title(f"Hierarchical View of {target_user}")
    plt.show()

def save_hierarchy_json(G, target_user, output_path):
    """Save the hierarchical structure as JSON."""
    hierarchy = nx.ego_graph(G, target_user, radius=2)

    hierarchy_data = {
        'target_user': target_user,
        'connections': list(hierarchy.edges)
    }

    with open(output_path, 'w') as f:
        json.dump(hierarchy_data, f, indent=4)

    print(f"Hierarchy saved to {output_path}")

# Main Execution
file_path = "/content/criminal_social_media_dataset.csv"  # Replace with your file path
target_user = input("Enter the name of the person to visualize their hierarchy: ").strip()
output_json = "hierarchy_output.json"

df = load_dataset(file_path)
G = build_network(df)
role_mapping = assign_roles(df)
visualize_hierarchy_tree(G, target_user, role_mapping)
save_hierarchy_json(G, target_user, output_json)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy in a tree-like structure with activity labels
def visualize_hierarchy_tree(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Role mapping for coloring nodes
    role_mapping = {}

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for first-level connections with activity labels
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            activity_type = following_row["Activity_Type"]

            # Assign role to first-level connections
            role_mapping[following] = assign_roles(second_level_interaction_count)

            G.add_edge(person_name, following, activity=activity_type)

            # Add edges for second-level connections (sub-connections) with activity labels
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    third_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]

                    # Assign role to second-level connections
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    G.add_edge(following, second_following, activity=third_level_activity_type)

    # Draw the graph using a hierarchical layout
    pos = graphviz_layout(G, prog="dot")  # Use Graphviz's dot layout for hierarchy

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    # Add edge labels (activity types)
    edge_labels = {(u, v): data['activity'] for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy_tree(person_name)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy in a tree-like structure with activity labels
def visualize_hierarchy_tree(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Role mapping for coloring nodes
    role_mapping = {}

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for first-level connections with activity labels
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            activity_type = following_row["Activity_Type"]

            # Assign role to first-level connections
            role_mapping[following] = assign_roles(second_level_interaction_count)

            G.add_edge(person_name, following, activity=activity_type)

            # Add edges for second-level connections (sub-connections) with activity labels
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    third_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]

                    # Assign role to second-level connections
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    G.add_edge(following, second_following, activity=third_level_activity_type)

    # Draw the graph using a hierarchical layout
    pos = graphviz_layout(G, prog="dot")  # Use Graphviz's dot layout for hierarchy

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    # Add edge labels (activity types)
    edge_labels = {(u, v): data['activity'] for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy_tree(person_name)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy in a tree-like structure with activity labels
def visualize_hierarchy_tree(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Role mapping for coloring nodes
    role_mapping = {}

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            activity_type = following_row["Activity_Type"]

            # Assign role to first-level connections
            role_mapping[following] = assign_roles(second_level_interaction_count)

            # Calculate score for first-level connection and add edge
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add edges for second-level connections (sub-connections)
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    third_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]

                    # Assign role to second-level connections
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    # Calculate score for second-level connection and add edge
                    score_second_level = second_level_interaction_count * activity_scores.get(third_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=third_level_activity_type)

    # Draw the graph using a hierarchical layout
    pos = graphviz_layout(G, prog="dot")  # Use Graphviz's dot layout for hierarchy

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    # Draw nodes and edges
    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    # Add edge labels (activity type and score)
    edge_labels = {(u, v): f"{data['weight']} ({data['activity']})" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Main loop to allow user input for specific persons
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    visualize_hierarchy_tree(person_name)


In [ ]:
!pip install tkinter

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from tkinter import Tk, Label, Entry, Button

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to visualize hierarchy in a tree-like structure with activity labels
def visualize_hierarchy_tree(person_name):
    # Create a directed graph
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return

    # Role mapping for coloring nodes
    role_mapping = {}

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Assign role to the main person
    role_mapping[person_name] = assign_roles(interaction_count)

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            second_level_interaction_count = following_row["Interaction_Count"]
            activity_type = following_row["Activity_Type"]

            # Assign role to first-level connections
            role_mapping[following] = assign_roles(second_level_interaction_count)

            # Calculate score for first-level connection and add edge
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add edges for second-level connections (sub-connections)
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    third_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    third_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]

                    # Assign role to second-level connections
                    role_mapping[second_following] = assign_roles(third_level_interaction_count)

                    # Calculate score for second-level connection and add edge
                    score_second_level = second_level_interaction_count * activity_scores.get(third_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=third_level_activity_type)

    # Draw the graph using a hierarchical layout
    pos = graphviz_layout(G, prog="dot")  # Use Graphviz's dot layout for hierarchy

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        role = role_mapping.get(node, "Weak Link")
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    # Draw nodes and edges
    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    # Add edge labels (activity type and score)
    edge_labels = {(u, v): f"{data['weight']} ({data['activity']})" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Function to handle input and hide input field after visualization
def handle_input():
    person_name = entry.get().strip()

    if not person_name:
        label.config(text="Please enter a valid name.")
        return

    visualize_hierarchy_tree(person_name)

    # Hide input field and button after visualization
    entry.pack_forget()
    button.pack_forget()
    label.config(text=f"Hierarchy visualization completed for '{person_name}'.")

# Create GUI using tkinter
root = Tk()
root.title("Hierarchy Visualization")

label = Label(root, text="Enter suspect's name:")
label.pack(pady=10)

entry = Entry(root)
entry.pack(pady=5)

button = Button(root, text="Visualize Hierarchy", command=handle_input)
button.pack(pady=10)

root.mainloop()


In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to visualize the graph interactively using Plotly
def visualize_interactive_hierarchy(G):
    if G is None or len(G.nodes) == 0:
        print("No hierarchy to display.")
        return

    # Extract edge data for visualization
    edge_x = []
    edge_y = []
    edge_labels = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_labels.append(f"{edge[2]['activity']} ({edge[2]['weight']})")

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []
    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"
        node_text.append(f"{node}<br>{role_text}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        text=[node for node in G.nodes()],
        marker=dict(
            size=10,
            color='blue',  # Default color (can be customized based on roles)
            line_width=2),
        hoverinfo='text',
        textfont=dict(size=10),
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    # Add callback for clicking on nodes
    fig.update_traces(
        marker=dict(size=20),
        selector=dict(mode='markers'),
        hovertemplate='%{text}<extra></extra>'
    )

    def click_callback(trace, points, state):
        clicked_node = points.customdata[points.point_inds[0]]
        print(f"Clicked on: {clicked_node}")

        # Build and visualize new hierarchy for clicked node
        new_hierarchy = build_hierarchy(clicked_node)
        visualize_interactive_hierarchy(new_hierarchy)

    fig.show()

# Main loop to allow user input and visualization
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    hierarchy_graph = build_hierarchy(person_name)

visualize_interactive_hierarchy(hierarchy_graph)


In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to visualize the graph using matplotlib
def visualize_hierarchy_tree(G, person_name):
    if G is None or len(G.nodes) == 0:
        print("No hierarchy to display.")
        return

    # Use Graphviz's dot layout for hierarchical visualization
    pos = graphviz_layout(G, prog="dot")

    plt.figure(figsize=(12, 8))

    # Assign colors based on roles
    node_colors = []
    for node in G.nodes():
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0] if node in df['User'].values else 0
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    # Draw nodes and edges
    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color='black', node_size=2000, font_size=8)

    # Add edge labels (activity type and score)
    edge_labels = {(u, v): f"{data['weight']} ({data['activity']})" for u, v, data in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.title(f"Hierarchical Connections of {person_name}")
    plt.show()

# Main loop to allow user input and dynamic exploration of hierarchies
while True:
    person_name = input("Enter the name of the suspect to visualize their hierarchy (or type 'exit' to quit): ").strip()

    if person_name.lower() == 'exit':
        print("Exiting program. Goodbye!")
        break

    hierarchy_graph = build_hierarchy(person_name)

    if hierarchy_graph is not None:
        visualize_hierarchy_tree(hierarchy_graph, person_name)


In [ ]:
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz

In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to visualize the graph interactively using Plotly
def visualize_interactive_hierarchy(G):
    if G is None or len(G.nodes) == 0:
        print("No hierarchy to display.")
        return

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")


In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        print(f"Person '{person_name}' not found in the dataset.")
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to visualize the graph interactively using Plotly
def visualize_interactive_hierarchy(G):
    if G is None or len(G.nodes) == 0:
        print("No hierarchy to display.")
        return

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color='blue',  # Default color (can be customized based on roles)
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    def click_callback(trace, points, state):
        clicked_node = points.customdata[points.point_inds[0]]
        print(f"Clicked on: {clicked_node}")

        # Build and visualize new hierarchy for clicked node
        new_hierarchy = build_hierarchy(clicked_node)
        visualize_interactive_hierarchy(new_hierarchy)
hierarchy_graph = build_hierarchy("Suraj Thakur")
visualize_interactive_hierarchy(hierarchy_graph)


In [ ]:
!pip install dash


In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color='blue',  # Default color (can be customized based on roles)
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name", debounce=True),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData")],
)
def update_graph(n_clicks, click_data):
    ctx = dash.callback_context

    if not ctx.triggered or n_clicks is None:
        return go.Figure()

    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]

import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color='blue',  # Default color (can be customized based on roles)
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name", debounce=True),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData")],
)
def update_graph(n_clicks, click_data):
    ctx = dash.callback_context

    if not ctx.triggered or n_clicks is None:
        return go.Figure()

    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]

    if triggered_id == "submit-button":
        person_name = ctx.states["input-person.value"]
        hierarchy_graph = build_hierarchy(person_name) #Corrected this line
        return create_figure(hierarchy_graph)
    elif triggered_id == "network-graph" and click_data:
        # Handle click data (if needed)
        pass

    return go.Figure()

if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color='blue',  # Default color (can be customized based on roles)
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    )
                )  # Corrected indentation

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("input-person", "value")],
)
def update_graph(n_clicks, input_value):
    if n_clicks is None:
      return create_figure(nx.DiGraph())

    person_name = input_value
    G = build_hierarchy(person_name)
    return create_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []
    node_colors = []  # List to store node colors

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

        # Assign colors based on roles
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0]
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color=node_colors,  # Use node_colors list for custom colors
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData"),
     Input("input-person", "value")],
     prevent_initial_call=True
)
def update_graph(n_clicks, click_data, input_person):
    ctx = dash.callback_context

    if ctx.triggered[0]["prop_id"] == "submit-button.n_clicks":
        person_name = input_person
        G = build_hierarchy(person_name)
    elif ctx.triggered[0]["prop_id"] == "network-graph.clickData":
        if click_data is not None:
            clicked_node = click_data['points'][0]['customdata'][0]
            G = build_hierarchy(clicked_node)
        else:
            return go.Figure()
    else:
        return go.Figure()

    return create_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []
    node_colors = []  # List to store node colors

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

        # Assign colors based on roles
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0]
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color=node_colors,  # Use node_colors list for custom colors
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData"),
     Input("input-person", "value")],
     prevent_initial_call=True
)
def update_graph(n_clicks, click_data, input_person):
    ctx = dash.callback_context

    if ctx.triggered[0]["prop_id"] == "submit-button.n_clicks":
        person_name = input_person
        G = build_hierarchy(person_name)
    elif ctx.triggered[0]["prop_id"] == "network-graph.clickData":
        if click_data is not None:
            clicked_node = click_data['points'][0]['customdata'][0]
            G = build_hierarchy(clicked_node)
        else:
            return go.Figure()
    else:
        return go.Figure()

    return create_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []
    node_colors = []  # List to store node colors

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

        # Assign colors based on roles
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0]
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color=node_colors,  # Use node_colors list for custom colors
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData"),
     Input("input-person", "value")],
     prevent_initial_call=True
)
def update_graph(n_clicks, click_data, input_person):
    ctx = dash.callback_context

    if ctx.triggered[0]["prop_id"] == "submit-button.n_clicks":
        person_name = input_person
        G = build_hierarchy(person_name)
    elif ctx.triggered[0]["prop_id"] == "network-graph.clickData":
        if click_data is not None:
            clicked_node = click_data['points'][0]['customdata'][0]
            G = build_hierarchy(clicked_node)
        else:
            return go.Figure()
    else:
        return go.Figure()

    return create_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for the graph
def create_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    pos = nx.spring_layout(G)  # Layout for nodes

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []
    node_colors = []  # List to store node colors

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

        # Assign colors based on roles
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0]
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color=node_colors,  # Use node_colors list for custom colors
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Hierarchical Network",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData")],
     [Input("input-person", "value")]
)
def update_graph(n_clicks, click_data, input_person):

    ctx = dash.callback_context

    if not ctx.triggered or n_clicks is None and click_data is None:
        return go.Figure()

    triggered_id = ctx.triggered[0]["prop_id"]

    if triggered_id == "submit-button.n_clicks" and input_person:
        G = build_hierarchy(input_person)

    elif triggered_id == "network-graph.clickData" and click_data is not None:
        clicked_node = click_data['points'][0]['customdata']
        G = build_hierarchy(clicked_node)

    else:
        return go.Figure()

    return create_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
import pandas as pd
import networkx as nx
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
from networkx.drawing.nx_agraph import graphviz_layout

# Load the dataset
df = pd.read_csv("criminal_social_media_dataset.csv")

# Scoring system based on activity type
activity_scores = {
    "Coordinating criminal activities": 10,
    "Spreading misinformation": 7,
    "Influencing normal people": 5,
    "Dark web transactions": 12,
    "Illegal trade communication": 15,
    "Recruiting Members": 8,
    "Fake News": 6,
    "Illegal Transactions": 10,
    "Suspicious Posting": 4
}

# Function to assign roles based on interaction count
def assign_roles(interaction_count):
    if interaction_count > 30:
        return "Leader"
    elif 15 <= interaction_count <= 30:
        return "Right-hand Man"
    elif 5 <= interaction_count < 15:
        return "Middleman"
    else:
        return "Weak Link"

# Function to build a directed graph for a given person
def build_hierarchy(person_name):
    G = nx.DiGraph()

    # Check if the person exists in the dataset
    if person_name not in df["User"].values:
        return None

    # Add the main person to the graph
    G.add_node(person_name)

    # Get the row for the main person
    person_row = df[df["User"] == person_name].iloc[0]
    followings = person_row["Following"].split(", ") if isinstance(person_row["Following"], str) else []
    interaction_count = person_row["Interaction_Count"]

    # Add edges for first-level connections
    for following in followings:
        if following in df["User"].values:
            following_row = df[df["User"] == following].iloc[0]
            activity_type = following_row["Activity_Type"]
            score = interaction_count * activity_scores.get(activity_type, 1)
            G.add_edge(person_name, following, weight=score, activity=activity_type)

            # Add second-level connections (sub-connections)
            second_level_followings = following_row["Following"].split(", ") if isinstance(following_row["Following"], str) else []
            for second_following in second_level_followings:
                if second_following in df["User"].values:
                    second_level_interaction_count = df[df["User"] == second_following]["Interaction_Count"].values[0]
                    second_level_activity_type = df[df["User"] == second_following]["Activity_Type"].values[0]
                    score_second_level = second_level_interaction_count * activity_scores.get(second_level_activity_type, 1)
                    G.add_edge(following, second_following, weight=score_second_level, activity=second_level_activity_type)

    return G

# Function to create a Plotly figure for hierarchical visualization using Graphviz's dot layout
def create_hierarchical_figure(G):
    if G is None or len(G.nodes) == 0:
        return go.Figure()

    # Use Graphviz's dot layout for hierarchical visualization
    pos = graphviz_layout(G, prog="dot")

    # Extract edge data for visualization
    edge_x = []
    edge_y = []

    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    # Extract node data for visualization
    node_x = []
    node_y = []

    node_text = []
    node_colors = []  # List to store node colors

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        role_text = f"Role: {assign_roles(df[df['User'] == node]['Interaction_Count'].values[0])}"

        node_text.append(f"{node}<br>{role_text}")

        # Assign colors based on roles
        interaction_count = df[df['User'] == node]['Interaction_Count'].values[0]
        role = assign_roles(interaction_count)
        if role == "Leader":
            node_colors.append("red")
        elif role == "Right-hand Man":
            node_colors.append("blue")
        elif role == "Middleman":
            node_colors.append("green")
        else:
            node_colors.append("gray")

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        textposition="top center",
        marker=dict(
            size=10,
            color=node_colors,  # Use node_colors list for custom colors
            line_width=2),
        hoverinfo='text',
        text=node_text,
        customdata=list(G.nodes())  # Attach node names as custom data for interactivity
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Hierarchical Network Visualization",
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[
                            dict(
                                text="Click on a node to explore its hierarchy",
                                showarrow=False,
                                xref="paper",
                                yref="paper",
                                x=0.5,
                                y=-0.1,
                                font=dict(size=12)
                            )
                        ]
                    ))

    return fig

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Label("Enter suspect's name:"),
        dcc.Input(id="input-person", type="text", placeholder="Enter name"),
        html.Button("Submit", id="submit-button"),
    ]),

    dcc.Graph(id="network-graph")
])

@app.callback(
    Output("network-graph", "figure"),
    [Input("submit-button", "n_clicks"),
     Input("network-graph", "clickData")],
     [Input("input-person", "value")]
)
def update_graph(n_clicks, click_data, input_person):

    ctx = dash.callback_context

    if not ctx.triggered or n_clicks is None and click_data is None:
        return go.Figure()

    triggered_id = ctx.triggered[0]["prop_id"]

    if triggered_id == "submit-button.n_clicks" and input_person:
        G = build_hierarchy(input_person)

    elif triggered_id == "network-graph.clickData" and click_data is not None:
        clicked_node = click_data['points'][0]['customdata']
        G = build_hierarchy(clicked_node)

    else:
        return go.Figure()

    return create_hierarchical_figure(G)

if __name__ == "__main__":
    app.run_server(debug=True)
